In [2]:
import os
from subprocess import check_output
import cv2

import numpy as np
import pandas as pd
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt

In [3]:
import numpy as np
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    sounds = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".wav")]
        for f in file_names:
            fs, dat = (wavfile.read(f))
            sounds.append((dat))
            labels.append(d)
    return sounds, labels

In [4]:
X,y = load_data('speech_commands_v0.01/')

/Users/bedabratachoudhury/anaconda2/lib/python2.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [5]:
X = np.array(X)

In [6]:
import sklearn
from sklearn.model_selection import train_test_split


In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 1)
NUM_CLASSES = 31

In [8]:
Xtest=np.array(([cv2.resize(image, (1, 16000)) for image in Xtest]))
Xtrain=np.array(([cv2.resize(image, (1, 16000)) for image in Xtrain]))

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D,ReLU,Dense,BatchNormalization,Dropout,Activation,MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [10]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Convert y labels to integers
le = LabelEncoder()
ytrain2 = le.fit_transform(ytrain)
ytest2 = le.fit_transform(ytest)

In [11]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add( LSTM( 512, input_shape=(16000, 1), return_sequences=True))
model.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
model.add(Dense(256))
model.add(BatchNormalization())
model.add( LSTM( 512, return_sequences=False))
model.add(Dense(31))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit( Xtrain, ytrain2, batch_size=100, epochs=20, shuffle = False)

Epoch 1/20
